# Pós-graduação Lato Sensu em Ciência de Dados e Big Data - PUC MG

## TCC turma 2020 - Gabriel Fonseca da Silva


### Objetivo: desenvolver um algoritmo de machine learning para prever a aprovação de um candidato na prova do ENCCEJA 2019 para obtenção do certificado do ensino fundamental

## Dados utilizados:

### a) Microdados do ENCCEJA Nacional Regular 2019 dos inscritos que fizeram as provas do ensino fundamental
##### https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/encceja

### b) Dados de homicídios por UF do ano de 2019 extraídos do DATASUS (grupo CID10 X85-Y09 e Y35-Y36)
#### http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sim/cnv/ext10uf.def

### c) População por UF estimada extraída do DATASUS (estimativas utilizadas pelo TCU)
#### http://tabnet.datasus.gov.br/cgi/tabcgi.exe?ibge/cnv/poptuf.def

### d) Índice de Desenvolvimeno Humano por Estado extraído do Atlas Brasil do IPEA
#### http://www.atlasbrasil.org.br/



## Aplicação: Tratamento dos dados

### 1) Importando bibliotecas


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

### 2) Processamento/Tratamento de Dados

In [2]:
# Carregando os dados

dados_tratados = pd.read_csv("02-dados_filtrados\dados_filtrados.csv", sep = ',', encoding = "ISO-8859-1")

print("\nDimensões: {0}".format(dados_tratados.shape))
print("\nCampos: {0}".format(dados_tratados.keys()))


Dimensões: (145815, 30)

Campos: Index(['NU_INSCRICAO', 'NU_IDADE', 'TP_SEXO', 'SG_UF_PROVA', 'IN_APROVADO_LC',
       'IN_APROVADO_CN', 'IN_APROVADO_CH', 'IN_APROVADO_MT', 'Q01', 'Q02',
       'Q03', 'Q04', 'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q15', 'Q16', 'Q17',
       'Q18', 'Q19', 'Q20', 'Q21', 'Q31', 'Q32', 'homicidios', 'Populacao',
       'taxapor100mil', 'idhm_2017'],
      dtype='object')


#### 2.1) Verificação de dados ausentes

In [3]:
# Verificação dos "missing values"

print(dados_tratados.isna().sum())

NU_INSCRICAO           0
NU_IDADE               1
TP_SEXO                0
SG_UF_PROVA            0
IN_APROVADO_LC         0
IN_APROVADO_CN         0
IN_APROVADO_CH         0
IN_APROVADO_MT         0
Q01                    0
Q02                    0
Q03                    0
Q04                    0
Q05                    0
Q06                    0
Q07                    0
Q08                    0
Q09                46270
Q15                46333
Q16                46333
Q17                46347
Q18                    0
Q19                    0
Q20                    0
Q21                    0
Q31                    0
Q32               145815
homicidios             0
Populacao              0
taxapor100mil          0
idhm_2017              0
dtype: int64


In [4]:
# Removendo registros dos "missing values" do atributo "NU_IDADE"

dados_tratados.dropna(subset=['NU_IDADE'], inplace=True)
print("\nDimensões: {0}".format(dados_tratados.shape))


Dimensões: (145814, 30)


In [5]:
# Removendo os atributos Q09, Q15, Q16, Q17 que possuem mais de 30% de missings e Q32 com 100% de missings  

del dados_tratados['Q09']
del dados_tratados['Q15']
del dados_tratados['Q16']
del dados_tratados['Q17']
del dados_tratados['Q32']

print("\nDimensões: {0}".format(dados_tratados.shape))


Dimensões: (145814, 25)


In [6]:
# Verificação dos "missing values"

print(dados_tratados.isna().sum())

NU_INSCRICAO      0
NU_IDADE          0
TP_SEXO           0
SG_UF_PROVA       0
IN_APROVADO_LC    0
IN_APROVADO_CN    0
IN_APROVADO_CH    0
IN_APROVADO_MT    0
Q01               0
Q02               0
Q03               0
Q04               0
Q05               0
Q06               0
Q07               0
Q08               0
Q18               0
Q19               0
Q20               0
Q21               0
Q31               0
homicidios        0
Populacao         0
taxapor100mil     0
idhm_2017         0
dtype: int64


#### 2.2) Verificação e exclusão de dados duplicados

In [7]:
# Criando o objeto "duplicados" a partir do objeto "dados" sem registros duplicados

duplicados = dados_tratados
duplicados.drop_duplicates()

# Verificando se havia dados duplicados

freq_dados = len(dados_tratados)
freq_duplicados = len(duplicados)


if freq_dados == freq_duplicados:
   print("Não existem registros duplicados")
else:
   print("Existem registros duplicados")


Não existem registros duplicados


#### 2.3) Verificação de inconsistências com a variável Q20 (série que deixou de estudar)

In [8]:
# Espera-se que não haja ninguém que deixou de estudar no ensino médio. Se houver é sinal de inconsistência.
# Os códigos J, K e L são respectivamente: 

# J - 1ª série do ensino médio (antigo 2º grau).
# K - 2ª série do ensino médio (antigo 2º grau).
# L - 3ª série do ensino médio (antigo 2º grau).

freq_Q20=dados_tratados['Q20'].value_counts().sort_index()

if (freq_Q20.J > 0) or (freq_Q20.K > 0) or (freq_Q20.L > 0):
    print("Existem inconsistências - quem deixou de estudar no ensino médio já possui ensino fundamental")
else:
    print("Não há incosnsitências")
    
print(freq_Q20)

Existem inconsistências - quem deixou de estudar no ensino médio já possui ensino fundamental
A     2850
B     1820
C     1277
D     3204
E    14343
F    24948
G    27581
H    34293
I    34441
J      645
K      186
L      226
Name: Q20, dtype: int64


In [9]:
# Excluindo os códigos J, K e L da base de dados

dados_tratados= dados_tratados[(dados_tratados.Q20 != 'J') & (dados_tratados.Q20 != 'K') & (dados_tratados.Q20 != 'L')]

dados_tratados['Q20'].value_counts().sort_index()


A     2850
B     1820
C     1277
D     3204
E    14343
F    24948
G    27581
H    34293
I    34441
Name: Q20, dtype: int64

#### 2.4) Criando a variável target

In [10]:
# Criando um campo que representa a soma do código das 4 provas. Se o resultado for 4 significa aprovação em todas.

dados_tratados['y']=(dados_tratados['IN_APROVADO_MT']+dados_tratados['IN_APROVADO_LC']+
                     dados_tratados['IN_APROVADO_CN']+dados_tratados['IN_APROVADO_CH'])


# Criando a variável target

dados_tratados['target'] = np.where(dados_tratados['y'] == 4, "Aprovado", "Reprovado")


In [11]:
pd.crosstab(dados_tratados['target'],dados_tratados['y'])

y           0.0    1.0    2.0    3.0    4.0
target                                     
Aprovado      0      0      0      0  63859
Reprovado  6872  12701  21081  40244      0

#### 2.5) Total de registros após o tratamento dos dados

In [12]:
# Dimensões

print("\nDimensões: {0}".format(dados_tratados.shape))


Dimensões: (144757, 27)


In [13]:
# Salvando os dados

dados_tratados.to_csv('03-dados_tratados/dados_tratados.csv',index=False)
